
**Diplomatura en Ciencia de Datos, Aprendizaje Automático y sus Aplicaciones**

**Edición 2021**

---
## Trabajo práctico entregable - Parte 1

In [1]:
import io
import matplotlib
import matplotlib.pyplot as plt
import numpy
import pandas as pd
import seaborn

seaborn.set_context('talk')
seaborn.set_theme(style='ticks')

## Lectura del dataset

En la notebook *00* se explican los detalles de la siguiente sección.

In [2]:
url = 'https://cs.famaf.unc.edu.ar/~mteruel/datasets/diplodatos/sysarmy_survey_2020_processed.csv'
df = pd.read_csv(url)

In [3]:
df[:3]

,profile_gender,profile_age,work_country,work_province,profile_years_experience,work_years_in_company,work_years_in_current_position,work_people_in_charge_of,profile_studies_level,profile_studies_level_state,...,work_has_violence_situations,profile_sexual_orientation,profile_has_disabilities,profile_has_disabilities_hiring_difficulties,company_employee_number,company_main_activity,company_recommended,company_diversity_policies,company_extra_benefits,company_best_companies_city
0,Mujer,26,Argentina,Ciudad Autónoma de Buenos Aires,3.0,3.0,3.0,0,Universitario,En curso,...,En mi trabajo actual,Homosexual,NaN,NaN,501-1000,Servicios / Consultoría de Software / Digital,7,2,"Capacitaciones y/o cursos, Comidas pagas / sub...",NaN
1,Hombre,29,Argentina,Corrientes,5.0,2.0,2.0,4,Universitario,En curso,...,Jamás,Heterosexual,Visual,No,201-500,Otras industrias,8,9,"Horarios flexibles, Stock options / RSUs, Viát...",NaN
2,Mujer,22,Argentina,Ciudad Autónoma de Buenos Aires,2.0,0.0,0.0,0,Secundario,Completado,...,En un trabajo anterior,Bisexual o queer,NaN,No,2001-5000,Otras industrias,6,9,"Clases de gimnasia online, Comidas pagas / sub...",NaN


# Ejercicio 2 - Densidades y varias variables

Responder a la pregunta general: **¿Qué herramientas (prácticas y/o teóricas) son útiles para explorar la base, y descubrir patrones o asociaciones?**

Para ello considere (igual al ejercicio anterior):

1. Seleccionar las columnas relevantes para analizar.

**Respuesta**
Dejando de lado los lenguajes de programación del ejercicio anterior, se procede a elegir las siguientes columnas
- Variables Numéricas: `salary_monthly_NETO`, `salary_monthly_BRUTO`, y `profile_age`.
- Variables Categóricas: `work_contract_type`, y `salary_in_usd`.
- Variables Ordinales: `profile_studies_level`, y `profile_studies_level_state` (también podría ser categórica).

2. Seleccionar las filas relevantes para analizar. Esto incluye la eliminación de valores extremos y erróneos, pero también puede enfocar el análisis en sub-poblaciones.

**Respuesta**
Observando los datos se pudo apreciar la existencia de valores *NaN* y erróneos, por lo que se procedió a eliminar los valores nulos y los menores a 10000 que existían en la columna `salary_monthly_NETO`.
Para los valores extremos superiores, se procedió a eliminar los mayores al percentil 99, entendiendo que esto genera un sesgo en la media muestral.
Un razonamiento idéntico se aplica para la columna `salary_monthly_BRUTO`.
En el caso de las columnas adicionales se procedió de la siguiente manera
- Se eliminan las filas con edades mayores a 100 años en `profile_age`, ya que carecen de sentido.
- Se aplica una transformación a los valores de `salary_in_usd`, para poder trabajar con valores *booleanos* en lugar de *strings*.

### 1. Selección de Columnas Relevantes

In [4]:
# Constantes Auxiliares.

# Numéricas
neto = 'salary_monthly_NETO'
bruto = 'salary_monthly_BRUTO'
edad = 'profile_age'
# Categóricas
contrato = 'work_contract_type'
dolares = 'salary_in_usd'
# Ordinales
nivel = 'profile_studies_level'
estado = 'profile_studies_level_state'

# Selección de Columnas Relevantes.
relevant_columns = [neto, bruto, edad, contrato, dolares, nivel, estado]

df.columns

Index(['profile_gender', 'profile_age', 'work_country', 'work_province',
       'profile_years_experience', 'work_years_in_company',
       'work_years_in_current_position', 'work_people_in_charge_of',
       'profile_studies_level', 'profile_studies_level_state',
       'profile_career', 'profile_university',
       'profile_specialization_courses', 'profile_open_source_contributions',
       'profile_programming_as_hobby', 'work_role', 'tools_platform',
       'tools_programming_languages', 'tools_frameworks', 'tools_data_bases',
       'tools_qa_testing', 'tools_IDEs', 'tools_work_pc_os',
       'tools_cellphone_os', 'work_on_call_duty', 'salary_on_call_duty_charge',
       'work_on_call_duty_charge_type', 'work_contract_type',
       'salary_monthly_BRUTO', 'salary_monthly_NETO', 'salary_in_usd',
       'salary_satisfaction', 'salary_comparison_last_semester',
       'salary_has_bonus', 'salary_bonus_tied_to',
       'salary_inflation_adjustment_2020',
       'salary_percentage_inf

### 2. Selección de Filas Relevantes

In [5]:
def filtrar_salarios(df, salario):
    """
    Dada una columna de salarios, aplica una serie de filtros al df.
    """

    # ¿Cuántos NaN existen en la columna de salarios?
    print(f'Cantidad de salarios nulos: {df[salario].isnull().sum()}')
    # Limpieza de valores nulos.
    df.dropna(subset=[salario], inplace=True)

    # Salario Mínimo para la subpoblación de estudio.
    min_salary = 10000
    # ¿Cuántos salarios erróneos existen en la columna de salarios?
    print(f'Cantidad de salarios <= 10K: {(df[salario] <= min_salary).sum()}')
    # Considerando el salario mínimo.
    df = df[df[salario] > min_salary]

    # Salario Máximo para la subpoblación de estudio (Percentil 99).
    max_salary = df[salario].quantile(0.99)
    print(f'Percentil 99: {max_salary}')
    # ¿Cuántos salarios son anómalos en la columna de salarios?
    print(f'Cantidad de salarios anómalos: {(df[salario] >= max_salary).sum()}')
    # Considerando el salario del percentil 99.
    df = df[df[salario] < max_salary]

    return df

In [6]:
print('Filtrado según salario BRUTO:')
df = filtrar_salarios(df, bruto)
print('----------')

print('Filtrado según salario NETO:')
df = filtrar_salarios(df, neto)
print('----------')

# Datos Muestrales (sobre el salario NETO y el salario BRUTO).
print(df[[neto, bruto]].describe())

Filtrado según salario BRUTO:
Cantidad de salarios nulos: 0
Cantidad de salarios <= 10K: 197
Percentil 99: 641939.2999999978
Cantidad de salarios anómalos: 59
----------
Filtrado según salario NETO:
Cantidad de salarios nulos: 197
Cantidad de salarios <= 10K: 61
Percentil 99: 400000.0
Cantidad de salarios anómalos: 59
----------
       salary_monthly_NETO  salary_monthly_BRUTO
count          5522.000000           5522.000000
mean          88651.609779         114537.589643
std           53566.195077          74585.652842
min           10700.000000          10602.410000
25%           54000.000000          64925.000000
50%           79000.000000          96009.000000
75%          105000.000000         140000.000000
max          396000.000000         609000.000000


In [7]:
# Edad Máxima para la subpoblación de estudio.
max_age = 100
# ¿Cuántas edades erróneas existen en la columna de edades?
print(f'Cantidad de edades >= 100: {(df[edad] >= max_age).sum()}')
# Considerando la edad máxima.
df = df[df[edad] < max_age]
print('----------')

# Datos Muestrales (sobre la edad).
print(df[edad].describe())

Cantidad de edades >= 100: 2
----------
count    5520.000000
mean       32.542572
std         7.257713
min        18.000000
25%        27.000000
50%        31.000000
75%        37.000000
max        67.000000
Name: profile_age, dtype: float64


In [8]:
print(f'Posibles valores de VA: {df[dolares].unique()}')
# Por razones de comodidad, queremos trabajar con valores booleanos en esta columna.
df[dolares] = df[dolares].notna()
print(f'Posibles valores de VA: {df[dolares].unique()}')
print('----------')

# Datos Muestrales (sobre el salario en dólares).
print(df[dolares].describe())

Posibles valores de VA: [nan 'Mi sueldo está dolarizado']
Posibles valores de VA: [False  True]
----------
count      5520
unique        2
top       False
freq       4992
Name: salary_in_usd, dtype: object


In [9]:
# Nos quedamos con las columnas que analizaremos.
df = df[relevant_columns]

print(f'Dimensiones del DF: {df.shape}')
df[:5]

Dimensiones del DF: (5520, 7)


,salary_monthly_NETO,salary_monthly_BRUTO,profile_age,work_contract_type,salary_in_usd,profile_studies_level,profile_studies_level_state
0,43000.0,53000.0,26,Full-Time,False,Universitario,En curso
1,63000.0,79000.0,29,Full-Time,False,Universitario,En curso
2,127000.0,160000.0,22,Full-Time,False,Secundario,Completado
3,102000.0,130000.0,39,Full-Time,False,Posgrado,Incompleto
4,106000.0,140000.0,32,Full-Time,False,Universitario,En curso


## a) Densidad conjunta

¿Qué herramientas visuales y modelos puede utilizar para estudiar la distribución y comportamiento de sus datos? 

Elija tres variables **numéricas** y dos variables **categóricas**. Visualice la base según varias de las variables elegidas. ¿Puede describir de alguna forma el comportamiento de sus datos? ¿Qué herramientas utilizaría? Describa.

## b) Asociación

Necesitamos decidir si sacar o no la columna de salario bruto, para hacer la encuesta más simple.
¿Existe una correlación entre el salario bruto y el salario neto? ¿Qué abordaje y medidas utilizaría?

## c) Densidad condicional 

Estudie la distribución del salario según el nivel de estudio.

Separe la población según el nivel de estudio (elija **dos** subpoblaciones numerosas), y grafique de manera comparativa ambos histogramas de la variable `salary_monthly_NETO`.
¿Considera que ambas variables son independientes?
¿Qué analizaría al respecto?

Calcule medidas de centralización y dispersión para cada subpoblación.

## d) Densidad conjunta condicional

Elija dos variables **numéricas** y una variable **categórica**.
Estudie la dispersión (*scatterplot*) de las dos variables numéricas, discriminando en color por la variable categórica (**hue** en *seaborn*).